In [1]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [45]:
# tables = ["Users", "Transactions", "Membership_Purchases", "MDR_Data", "User_Activity"]
# file_paths = {}
# for table in tables:
#     file_path = f"/data/{table}.csv"

#     file_paths[table] = file_path

users = pd.read_csv('data/users.csv')
transactions = pd.read_csv('data/transactions.csv')
memberships = pd.read_csv('data/membership_purchases.csv')
mdr_data = pd.read_csv('data/mdr_data.csv')
user_activity = pd.read_csv('data/user_activity.csv')

print(users.head())

   User_ID     Name                Email         Phone
0      101    Alice    alice@example.com  1.234568e+11
1      102      Bob      bob@example.com  1.234568e+11
2      103  Charlie  charlie@example.com  1.234568e+11
3      104    David    david@example.com  1.234568e+11
4      105      Eve      eve@example.com  1.234568e+11


In [46]:
# Get the last mdr based on membership type
mdr_data = mdr_data.tail(2)
mdr_data

,MDR_ID,Membership_Type,MDR_Percentage
2,3,Basic,3.0
3,4,Premium,1.0


In [10]:
transactions.head()

,Transaction_ID,User_ID,Transaction_Amount,Timestamp
0,1,101,50.0,2024-03-17 10:05:23
1,2,102,75.0,2024-03-17 12:30:45
2,3,103,100.0,2024-03-17 15:20:10
3,4,104,120.0,2024-03-18 09:45:35
4,5,105,90.0,2024-03-18 11:15:22


In [11]:
memberships.head()

,Membership_ID,User_ID,Membership_Type,Purchase_Date,Expiry_Date
0,1,101,Premium,2024-01-01,2024-02-01
1,2,102,Basic,2024-02-15,2024-03-15
2,3,103,Basic,2024-03-05,2024-04-05
3,4,104,Premium,2024-03-10,2024-04-10
4,5,105,Basic,2024-03-15,2024-04-15


In [12]:
# fill 0 for the null transactios amount
transactions.fillna(0, inplace=True)
memberships.dropna(subset=['Membership_Type'], inplace=True)

transactions.head()

,Transaction_ID,User_ID,Transaction_Amount,Timestamp
0,1,101,50.0,2024-03-17 10:05:23
1,2,102,75.0,2024-03-17 12:30:45
2,3,103,100.0,2024-03-17 15:20:10
3,4,104,120.0,2024-03-18 09:45:35
4,5,105,90.0,2024-03-18 11:15:22


In [18]:
# transform columns date
transactions['Timestamp'] = pd.to_datetime(transactions['Timestamp'])
memberships['Purchase_Date'] = pd.to_datetime(memberships['Purchase_Date'])
memberships['Expiry_Date'] = pd.to_datetime(memberships['Expiry_Date'])
user_activity['Activity_Date'] = pd.to_datetime(user_activity['Activity_Date'])

transactions.head()

,Transaction_ID,User_ID,Transaction_Amount,Timestamp
0,1,101,50.0,2024-03-17 10:05:23
1,2,102,75.0,2024-03-17 12:30:45
2,3,103,100.0,2024-03-17 15:20:10
3,4,104,120.0,2024-03-18 09:45:35
4,5,105,90.0,2024-03-18 11:15:22


In [37]:
memberships['Current_Status'] = memberships.apply(
    lambda row: 'Active' if pd.Timestamp.now() <= row['Expiry_Date'] else 'Expired',
    axis=1
)

memberships.head()

,Membership_ID,User_ID,Membership_Type,Purchase_Date,Expiry_Date,Current_Status
0,1,101,Premium,2024-01-01,2024-02-01,Expired
1,2,102,Basic,2024-02-15,2024-03-15,Expired
2,3,103,Basic,2024-03-05,2024-04-05,Expired
3,4,104,Premium,2024-03-10,2024-04-10,Expired
4,5,105,Basic,2024-03-15,2024-04-15,Expired


In [ ]:
# Join users to membership, and transaction to memberships
users_membership_df = pd.merge(users, memberships, on='User_ID', how='left')
users_membership_df.head()

15


In [25]:
transactions_df = pd.merge(
            transactions,
            users_membership_df,
            on='User_ID',
            how='left'
        )
print(len(transactions_df))
transactions_df.head()


15


,Transaction_ID,User_ID,Transaction_Amount,Timestamp,Name,Email,Phone,Membership_ID,Membership_Type,Purchase_Date,Expiry_Date
0,1,101,50.0,2024-03-17 10:05:23,Alice,alice@example.com,1.234568e+11,1,Premium,2024-01-01,2024-02-01
1,2,102,75.0,2024-03-17 12:30:45,Bob,bob@example.com,1.234568e+11,2,Basic,2024-02-15,2024-03-15
2,3,103,100.0,2024-03-17 15:20:10,Charlie,charlie@example.com,1.234568e+11,3,Basic,2024-03-05,2024-04-05
3,4,104,120.0,2024-03-18 09:45:35,David,david@example.com,1.234568e+11,4,Premium,2024-03-10,2024-04-10
4,5,105,90.0,2024-03-18 11:15:22,Eve,eve@example.com,1.234568e+11,5,Basic,2024-03-15,2024-04-15


In [28]:
# Add MDR percentage to transactions
transactions_with_mdr_df = pd.merge(
    transactions_df,
    mdr_data,
    on='Membership_Type',
    how='left'
)

# print(len(transactions_with_mdr_df))
transactions_with_mdr_df.head()

,Transaction_ID,User_ID,Transaction_Amount,Timestamp,Name,Email,Phone,Membership_ID,Membership_Type,Purchase_Date,Expiry_Date,MDR_ID,MDR_Percentage
0,1,101,50.0,2024-03-17 10:05:23,Alice,alice@example.com,1.234568e+11,1,Premium,2024-01-01,2024-02-01,4.0,1.0
1,2,102,75.0,2024-03-17 12:30:45,Bob,bob@example.com,1.234568e+11,2,Basic,2024-02-15,2024-03-15,3.0,3.0
2,3,103,100.0,2024-03-17 15:20:10,Charlie,charlie@example.com,1.234568e+11,3,Basic,2024-03-05,2024-04-05,3.0,3.0
3,4,104,120.0,2024-03-18 09:45:35,David,david@example.com,1.234568e+11,4,Premium,2024-03-10,2024-04-10,4.0,1.0
4,5,105,90.0,2024-03-18 11:15:22,Eve,eve@example.com,1.234568e+11,5,Basic,2024-03-15,2024-04-15,3.0,3.0


In [32]:
# Aggregate daily activity reports
daily_activity_report = transactions_with_mdr_df.groupby(['User_ID', 'Name', 'Timestamp', 'Membership_Type']).agg(
    First_Transaction=('Timestamp', 'min'),
    Invoice_Total_Count=('Transaction_ID', 'count'),
    Invoice_Total_Amount=('Transaction_Amount', 'sum')
).reset_index()

daily_activity_report.head()

,User_ID,Name,Timestamp,Membership_Type,First_Transaction,Invoice_Total_Count,Invoice_Total_Amount
0,101,Alice,2024-03-17 10:05:23,Premium,2024-03-17 10:05:23,1,50.0
1,102,Bob,2024-03-17 12:30:45,Basic,2024-03-17 12:30:45,1,75.0
2,103,Charlie,2024-03-17 15:20:10,Basic,2024-03-17 15:20:10,1,100.0
3,104,David,2024-03-18 09:45:35,Premium,2024-03-18 09:45:35,1,120.0
4,105,Eve,2024-03-18 11:15:22,Basic,2024-03-18 11:15:22,1,90.0


In [40]:
# Group transactions by date and membership type to calculate totals
transaction_totals = transactions_with_mdr_df.groupby(
    [transactions_with_mdr_df['Timestamp'].dt.date, 'Membership_Type']
).agg(
    total_invoice_count=('Transaction_ID', 'count'),
    total_transaction_amount=('Transaction_Amount', 'sum')
).reset_index()

transaction_totals.head()

,Timestamp,Membership_Type,total_invoice_count,total_transaction_amount
0,2024-03-17,Basic,2,175.0
1,2024-03-17,Premium,1,50.0
2,2024-03-18,Basic,2,150.0
3,2024-03-18,Premium,1,120.0
4,2024-03-19,Premium,2,80.0


In [ ]:
# Format the report
daily_activity_report['Date'] = daily_activity_report['Timestamp'].dt.date

In [35]:
# Group transactions by date and calculate totals
transaction_breakdown = transactions_with_mdr_df.groupby(daily_activity_report['Timestamp'].dt.date).agg(
    total_invoice_count=('Transaction_ID', 'count'),
    total_transaction_amount=('Transaction_Amount', 'sum'),
).reset_index()

transaction_breakdown

,Timestamp,total_invoice_count,total_transaction_amount
0,2024-03-17,3,225.0
1,2024-03-18,3,270.0
2,2024-03-19,2,110.0
3,2024-03-20,2,150.0
4,2024-03-21,2,120.0
5,2024-03-22,2,150.0


In [ ]:
# Apply the MDR percentage to calculate the effective amount for each transaction
transactions_with_mdr_df['Effective_Amount'] = transactions_with_mdr_df['Transaction_Amount'] * (1 - transactions_with_mdr_df['MDR_Percentage'] / 100)

transactions_with_mdr_df.head()

,Transaction_ID,User_ID,Transaction_Amount,Timestamp,Name,Email,Phone,Membership_ID,Membership_Type,Purchase_Date,Expiry_Date,MDR_ID,MDR_Percentage,Effective_Amount
0,1,101,50.0,2024-03-17 10:05:23,Alice,alice@example.com,1.234568e+11,1,Premium,2024-01-01,2024-02-01,4.0,1.0,49.50
1,2,102,75.0,2024-03-17 12:30:45,Bob,bob@example.com,1.234568e+11,2,Basic,2024-02-15,2024-03-15,3.0,3.0,72.75
2,3,103,100.0,2024-03-17 15:20:10,Charlie,charlie@example.com,1.234568e+11,3,Basic,2024-03-05,2024-04-05,3.0,3.0,97.00
3,4,104,120.0,2024-03-18 09:45:35,David,david@example.com,1.234568e+11,4,Premium,2024-03-10,2024-04-10,4.0,1.0,118.80
4,5,105,90.0,2024-03-18 11:15:22,Eve,eve@example.com,1.234568e+11,5,Basic,2024-03-15,2024-04-15,3.0,3.0,87.30


In [55]:
# fill 0 for the null transactios amount
transactions.fillna(0, inplace=True)
memberships.dropna(subset=['Membership_Type'], inplace=True)

# transform columns date
transactions['Timestamp'] = pd.to_datetime(transactions['Timestamp'])
memberships['Purchase_Date'] = pd.to_datetime(memberships['Purchase_Date'])
memberships['Expiry_Date'] = pd.to_datetime(memberships['Expiry_Date'])
user_activity['Activity_Date'] = pd.to_datetime(user_activity['Activity_Date'])

# Join table users and membership
users_membership_df = pd.merge(users, memberships, on='User_ID', how='left')

# Join users membership and transactions
transactions_df = pd.merge(transactions, users_membership_df, on='User_ID', how='left')

# Add MDR percentage to transactions
transactions_with_mdr_df = pd.merge(transactions_df, mdr_data, on='Membership_Type', how='left')

# Status membership when purchase
transactions_with_mdr_df['Membership_Purchase_Status'] = transactions_with_mdr_df.apply(
    lambda row: 'Active' if row['Purchase_Date'] <= row['Expiry_Date'] else 'Expired',
    axis=1
)

transactions_with_mdr_df.head()

,Transaction_ID,User_ID,Transaction_Amount,Timestamp,Name,Email,Phone,Membership_ID,Membership_Type,Purchase_Date,Expiry_Date,MDR_ID,MDR_Percentage,Membership_Purchase_Status
0,1,101,50.0,2024-03-17 10:05:23,Alice,alice@example.com,1.234568e+11,1.0,Premium,2024-01-01,2024-02-01,4.0,1.0,Active
1,2,102,75.0,2024-03-17 12:30:45,Bob,bob@example.com,1.234568e+11,2.0,Basic,2024-02-15,2024-03-15,3.0,3.0,Active
2,3,103,100.0,2024-03-17 15:20:10,Charlie,charlie@example.com,1.234568e+11,3.0,Basic,2024-03-05,2024-04-05,3.0,3.0,Active
3,4,104,120.0,2024-03-18 09:45:35,David,david@example.com,1.234568e+11,4.0,Premium,2024-03-10,2024-04-10,4.0,1.0,Active
4,5,105,90.0,2024-03-18 11:15:22,Eve,eve@example.com,1.234568e+11,5.0,Basic,2024-03-15,2024-04-15,3.0,3.0,Active


In [56]:
# User profiling
# Aggregate user profiling
transactions_with_mdr_df.loc[
    transactions_with_mdr_df['Membership_Purchase_Status'] == 'Active', 'Effective_Amount'
] = transactions_with_mdr_df['Transaction_Amount'] * (
    1 - transactions_with_mdr_df['MDR_Percentage'] / 100
)

transactions_with_mdr_df.head()

,Transaction_ID,User_ID,Transaction_Amount,Timestamp,Name,Email,Phone,Membership_ID,Membership_Type,Purchase_Date,Expiry_Date,MDR_ID,MDR_Percentage,Membership_Purchase_Status,Effective_Amount
0,1,101,50.0,2024-03-17 10:05:23,Alice,alice@example.com,1.234568e+11,1.0,Premium,2024-01-01,2024-02-01,4.0,1.0,Active,49.50
1,2,102,75.0,2024-03-17 12:30:45,Bob,bob@example.com,1.234568e+11,2.0,Basic,2024-02-15,2024-03-15,3.0,3.0,Active,72.75
2,3,103,100.0,2024-03-17 15:20:10,Charlie,charlie@example.com,1.234568e+11,3.0,Basic,2024-03-05,2024-04-05,3.0,3.0,Active,97.00
3,4,104,120.0,2024-03-18 09:45:35,David,david@example.com,1.234568e+11,4.0,Premium,2024-03-10,2024-04-10,4.0,1.0,Active,118.80
4,5,105,90.0,2024-03-18 11:15:22,Eve,eve@example.com,1.234568e+11,5.0,Basic,2024-03-15,2024-04-15,3.0,3.0,Active,87.30


In [57]:
user_profiling = transactions_with_mdr_df.groupby(['User_ID', 'Name', 'Timestamp', 'Membership_Type', 'Membership_Purchase_Status', 'MDR_Percentage']).agg(
    First_Transaction=('Timestamp', 'min'),
    Invoice_Total_Count=('Transaction_ID', 'count'),
    Invoice_Total_Amount=('Transaction_Amount', 'sum')
).reset_index()

user_profiling

,User_ID,Name,Timestamp,Membership_Type,Membership_Purchase_Status,MDR_Percentage,First_Transaction,Invoice_Total_Count,Invoice_Total_Amount
0,101,Alice,2024-03-17 10:05:23,Premium,Active,1.0,2024-03-17 10:05:23,1,50.0
1,102,Bob,2024-03-17 12:30:45,Basic,Active,3.0,2024-03-17 12:30:45,1,75.0
2,103,Charlie,2024-03-17 15:20:10,Basic,Active,3.0,2024-03-17 15:20:10,1,100.0
3,104,David,2024-03-18 09:45:35,Premium,Active,1.0,2024-03-18 09:45:35,1,120.0
4,105,Eve,2024-03-18 11:15:22,Basic,Active,3.0,2024-03-18 11:15:22,1,90.0
5,106,Frank,2024-03-18 13:30:00,Basic,Active,3.0,2024-03-18 13:30:00,1,60.0
6,107,Greg,2024-03-19 09:00:00,Premium,Active,1.0,2024-03-19 09:00:00,1,0.0
7,109,Ivy,2024-03-19 14:45:00,Premium,Active,1.0,2024-03-19 14:45:00,1,80.0
8,110,Jack,2024-03-20 10:20:00,Basic,Active,3.0,2024-03-20 10:20:00,1,70.0
9,111,Kate,2024-03-20 13:30:00,Premium,Active,1.0,2024-03-20 13:30:00,1,65.0


In [58]:
# Group transactions by date and membership type to calculate totals
transaction_totals = transactions_with_mdr_df.groupby(
    [transactions_with_mdr_df['Timestamp'].dt.date, 'Membership_Type']
).agg(
    total_invoice_count=('Transaction_ID', 'count'),
    total_transaction_amount=('Transaction_Amount', 'sum')
).reset_index()

transaction_totals

,Timestamp,Membership_Type,total_invoice_count,total_transaction_amount
0,2024-03-17,Basic,2,175.0
1,2024-03-17,Premium,1,50.0
2,2024-03-18,Basic,2,150.0
3,2024-03-18,Premium,1,120.0
4,2024-03-19,Premium,2,80.0
5,2024-03-20,Basic,1,70.0
6,2024-03-20,Premium,1,65.0
7,2024-03-21,Basic,2,100.0
8,2024-03-22,Basic,1,0.0
9,2024-03-22,Premium,1,105.0


In [59]:
# Group transactions by date and calculate totals
transaction_breakdown = transactions_with_mdr_df.groupby(daily_activity_report['Timestamp'].dt.date).agg(
    total_invoice_count=('Transaction_ID', 'count'),
    total_transaction_amount=('Transaction_Amount', 'sum'),
).reset_index()

transaction_breakdown

,Timestamp,total_invoice_count,total_transaction_amount
0,2024-03-17,3,225.0
1,2024-03-18,3,270.0
2,2024-03-19,2,110.0
3,2024-03-20,2,150.0
4,2024-03-21,2,120.0
5,2024-03-22,2,150.0
